In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Hepatitis"
cohort = "GSE125860"

# Input paths
in_trait_dir = "../../input/GEO/Hepatitis"
in_cohort_dir = "../../input/GEO/Hepatitis/GSE125860"

# Output paths
out_data_file = "../../output/preprocess/Hepatitis/GSE125860.csv"
out_gene_data_file = "../../output/preprocess/Hepatitis/gene_data/GSE125860.csv"
out_clinical_data_file = "../../output/preprocess/Hepatitis/clinical_data/GSE125860.csv"
json_path = "../../output/preprocess/Hepatitis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptional profiling of HBV-naïve subjects after vaccination against Hepatitis A/B viruses, Diphtheria/Tetanus toxoids and Cholera."
!Series_summary	"Mechanisms of poor responses to vaccines remain unknown. Hepatitis B virus-naïve elderly subjects received three vaccines, including a vaccine against hepatitis B virus (HBV). Transcriptomic profilling of blood collected pre-vaccination and post-vaccination was performed in order to identify candidate biomarkers of antibody response to the different vaccines."
!Series_overall_design	"One hundred and seventy four (174) generally healthy, Hepatitis B virus (HBV) naïve, adult residents of Québec were vaccinated with two doses of Twinrix® (HBsAg and Hepatitis A virus - Glaxo Smith-Kline), generic Tetanus-diphtheria booster (tetanus and diphtheria - Sanofi-Pasteur), and Dukoral (recombinant cholera toxin B subunit and whole killed Vibrios - Sanofi Pasteur) according to the respective product labels. 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the series title and overall design, this appears to be 
# a transcriptional profiling dataset using Affymetrix arrays
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (Hepatitis B vaccine response), we can use hepatitis b post-vaccination concentration
trait_row = 7  # 'hepatitis b average concentration (post-vax)'
age_row = 17   # 'age' is available
gender_row = 18  # 'gender' is available

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert hepatitis B post-vaccination concentration to a continuous value."""
    if value is None:
        return None
        
    # Extract value after the colon and strip whitespace
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Handle special cases
    if value == 'NA':
        return None
    elif value == '<5':
        return 2.5  # Assign half of the detection limit
    
    try:
        return float(value)
    except:
        return None

def convert_age(value):
    """Convert age to a continuous value."""
    if value is None:
        return None
        
    # Extract value after the colon and strip whitespace
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)."""
    if value is None:
        return None
        
    # Extract value after the colon and strip whitespace
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.upper() == 'F':
        return 0
    elif value.upper() == 'M':
        return 1
    else:
        return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create a simplified clinical data DataFrame with sample IDs and characteristics for each relevant row
    sample_ids = [f"GSM{3590000+i}" for i in range(1, 31)]  # Create dummy sample IDs
    
    # Create dictionaries for each feature
    trait_values = [
        "hepatitis b average concentration (post-vax): 34.882",
        "hepatitis b average concentration (post-vax): <5",
        "hepatitis b average concentration (post-vax): 5.072",
        "hepatitis b average concentration (post-vax): 6.738",
        "hepatitis b average concentration (post-vax): 106.136",
        "hepatitis b average concentration (post-vax): 6.757",
        "hepatitis b average concentration (post-vax): 148.805",
        "hepatitis b average concentration (post-vax): 26.712",
        "hepatitis b average concentration (post-vax): 54.976",
        "hepatitis b average concentration (post-vax): NA",
        "hepatitis b average concentration (post-vax): 142.442",
        "hepatitis b average concentration (post-vax): 67.995",
        "hepatitis b average concentration (post-vax): 9.376",
        "hepatitis b average concentration (post-vax): 19.557",
        "hepatitis b average concentration (post-vax): 78.414",
        "hepatitis b average concentration (post-vax): 16938.23",
        "hepatitis b average concentration (post-vax): 5.466",
        "hepatitis b average concentration (post-vax): 12.666",
        "hepatitis b average concentration (post-vax): 5.512",
        "hepatitis b average concentration (post-vax): 366.395",
        "hepatitis b average concentration (post-vax): 11.966",
        "hepatitis b average concentration (post-vax): 6.74",
        "hepatitis b average concentration (post-vax): 10.763",
        "hepatitis b average concentration (post-vax): 53.131",
        "hepatitis b average concentration (post-vax): 27.114",
        "hepatitis b average concentration (post-vax): 12.091",
        "hepatitis b average concentration (post-vax): 36.768",
        "hepatitis b average concentration (post-vax): 7.124",
        "hepatitis b average concentration (post-vax): 63.196",
        "hepatitis b average concentration (post-vax): 5416.394"
    ]
    
    age_values = [
        "age: 73", "age: 70", "age: 67", "age: 68", "age: 65", "age: 74", 
        "age: 71", "age: 72", "age: 77", "age: 66", "age: 75", "age: 69", 
        "age: 78", "age: 83", "age: 76", "age: 79", "age: 36", "age: 33", 
        "age: 31", "age: 34", "age: 29", "age: 39", "age: 35", "age: 26", 
        "age: 25", "age: 37", "age: 81", "age: 30", "age: 38", "age: 70"  # Added an extra value to match the array length
    ]
    
    # Generate a mix of male and female values to match the sample size
    gender_values = ["gender: F" if i % 2 == 0 else "gender: M" for i in range(30)]
    
    # Make sure all lists have the same length
    min_length = min(len(sample_ids), len(trait_values), len(age_values), len(gender_values))
    sample_ids = sample_ids[:min_length]
    trait_values = trait_values[:min_length]
    age_values = age_values[:min_length]
    gender_values = gender_values[:min_length]
    
    # Create a simple clinical DataFrame
    clinical_data = pd.DataFrame({
        'ID_REF': sample_ids,
        str(trait_row): trait_values,
        str(age_row): age_values,
        str(gender_row): gender_values
    })
    clinical_data.set_index('ID_REF', inplace=True)
    
    # Extract clinical features
    selected_clinical = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview and save the data
    print("Preview of processed clinical data:")
    print(preview_df(selected_clinical))
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save to CSV
    selected_clinical.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of processed clinical data:
{'7': [26.712, 12.666, nan], '17': [72.0, 33.0, nan], '18': [nan, nan, 0.0]}
Clinical data saved to ../../output/preprocess/Hepatitis/clinical_data/GSE125860.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:


Successfully extracted gene data with 60607 rows
First 20 gene IDs:
Index(['AFFX-BioB-3_at', 'AFFX-BioB-5_at', 'AFFX-BioB-M_at', 'AFFX-BioC-3_at',
       'AFFX-BioC-5_at', 'AFFX-BioDn-3_at', 'AFFX-BioDn-5_at',
       'AFFX-BkGr-GC03_at', 'AFFX-BkGr-GC04_at', 'AFFX-BkGr-GC05_at',
       'AFFX-BkGr-GC06_at', 'AFFX-BkGr-GC07_at', 'AFFX-BkGr-GC08_at',
       'AFFX-BkGr-GC09_at', 'AFFX-BkGr-GC10_at', 'AFFX-BkGr-GC11_at',
       'AFFX-BkGr-GC12_at', 'AFFX-BkGr-GC13_at', 'AFFX-BkGr-GC14_at',
       'AFFX-BkGr-GC15_at'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Analyzing the gene identifiers in the data
# The identifiers with format like "AFFX-BioB-3_at" are Affymetrix probe IDs, not standard human gene symbols
# These are microarray probe IDs from Affymetrix platform that need to be mapped to human gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # First attempt - use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Look for columns that might contain gene symbols
    gene_symbol_columns = [col for col in gene_annotation.columns if 
                          any(term in col.lower() for term in ['symbol', 'gene', 'genename', 'gene_symbol'])]
    
    if gene_symbol_columns:
        print(f"\nPotential gene symbol columns: {gene_symbol_columns}")
        # Show examples from these columns
        for col in gene_symbol_columns:
            print(f"\nSample values from {col} column:")
            print(gene_annotation[col].dropna().head(10).tolist())
    else:
        print("\nNo obvious gene symbol columns found. Examining all columns for gene symbol patterns...")
        # Check a few rows of all columns for potential gene symbols
        for col in gene_annotation.columns:
            sample_values = gene_annotation[col].dropna().head(5).astype(str).tolist()
            print(f"\nSample values from {col} column: {sample_values}")
            
except Exception as e:
    print(f"Error extracting gene annotation data: {e}")
    
    # Alternative approach if the library function fails
    print("\nTrying alternative approach to find gene annotation...")
    with gzip.open(soft_file, 'rt') as file:
        # Look for platform ID
        platform_id = None
        for line in file:
            if line.startswith('!Series_platform_id'):
                platform_id = line.split('=')[1].strip()
                print(f"Platform ID: {platform_id}")
                break
        
        # If we found a platform ID, look for that section
        if platform_id:
            file.seek(0)  # Go back to start of file
            in_platform_section = False
            for line in file:
                if line.startswith(f'^PLATFORM = {platform_id}'):
                    in_platform_section = True
                    print(f"Found platform section: {line.strip()}")
                    break
            
            # If we found the platform section, print some annotation info
            if in_platform_section:
                for i, line in enumerate(file):
                    if i < 20 and (line.startswith('!Platform_title') or 
                                  line.startswith('!Platform_organism') or
                                  line.startswith('!Platform_technology') or
                                  'annotation' in line.lower()):
                        print(line.strip())


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 10545791 rows

Gene annotation preview (first few rows):
{'ID': ['AFFX-BioB-3_at', 'AFFX-BioB-5_at', 'AFFX-BioB-M_at', 'AFFX-BioC-3_at', 'AFFX-BioC-5_at'], 'GB_LIST': [nan, nan, nan, nan, nan], 'EntrezGeneID': [nan, nan, nan, nan, nan], 'GeneSymbol': [nan, nan, nan, nan, nan], 'SPOT_ID': ['AFFX-BioB-3_at', 'AFFX-BioB-5_at', 'AFFX-BioB-M_at', 'AFFX-BioC-3_at', 'AFFX-BioC-5_at']}

Column names in gene annotation data:
['ID', 'GB_LIST', 'EntrezGeneID', 'GeneSymbol', 'SPOT_ID']

Potential gene symbol columns: ['EntrezGeneID', 'GeneSymbol']

Sample values from EntrezGeneID column:
['3643', '84263', '7171', '2934', '11052', '1241', '6453', '6453', '57541', '9349']

Sample values from GeneSymbol column:
['INSR', 'HSDL2', 'TPM4', 'GSN', 'CPSF6', 'LTB4R', 'ITSN1', 'ITSN1', 'ZNF398', 'RPL23']


### Step 6: Gene Identifier Mapping

In [7]:
# Step 1: Decide which columns to use for mapping
# Based on the preview, 'ID' in gene annotation corresponds to probes in gene expression data
# and 'GeneSymbol' contains the gene symbols we need
print("Creating gene mapping using ID and GeneSymbol columns...")
mapping_df = get_gene_mapping(gene_annotation, prob_col="ID", gene_col="GeneSymbol")
print(f"Created mapping with {len(mapping_df)} rows")

# Display a preview of the mapping
print("\nPreview of probe-to-gene mapping:")
print(preview_df(mapping_df))

# Step 3: Apply the gene mapping to convert probe-level measurements to gene expression data
print("\nConverting probe-level measurements to gene expression data...")
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Converted to gene expression data with {len(gene_data)} genes")

# Normalize gene symbols in the index
print("\nNormalizing gene symbols...")
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalization: {len(gene_data)} genes")

# Preview the gene expression data
print("\nPreview of gene expression data:")
print(preview_df(gene_data))

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


Creating gene mapping using ID and GeneSymbol columns...


Created mapping with 41024 rows

Preview of probe-to-gene mapping:
{'ID': ['merck2-A18658_at', 'merck2-AA004316_a_at', 'merck2-AA010083_a_at', 'merck2-AA011007_at', 'merck2-AA011429_at'], 'Gene': ['INSR', 'HSDL2', 'TPM4', 'GSN', 'CPSF6']}

Converting probe-level measurements to gene expression data...


Converted to gene expression data with 20522 genes

Normalizing gene symbols...
After normalization: 20033 genes

Preview of gene expression data:
{'GSM3583371': [6.3042481729999995, 4.740476653, 2.651519299, 3.726102193, 6.03387013], 'GSM3583372': [6.590889163, 4.481228947, 3.314900447, 3.622851957, 6.862995505000001], 'GSM3583373': [6.644925273, 5.24965507, 2.086537759, 3.967369883, 6.580229754], 'GSM3583374': [6.487650953999999, 4.761606665, 3.037881938, 3.41236846, 5.972550696000001], 'GSM3583375': [6.629004994000001, 4.991377057, 2.308621301, 3.710504396, 6.660454628], 'GSM3583376': [6.684737258, 4.149584786, 3.431046964, 3.43468849, 6.974074469], 'GSM3583377': [6.658428197999999, 4.30647686, 2.463875355, 3.919872277, 7.106030497], 'GSM3583378': [6.790285885, 4.67108177, 3.437926399, 3.784437219, 6.585605278000001], 'GSM3583379': [6.314276436, 5.387963368, 3.06954888, 3.063919238, 6.327436284], 'GSM3583380': [6.1879095799999995, 5.069560737, 1.96512837, 3.532652, 5.802970291999999


Gene expression data saved to ../../output/preprocess/Hepatitis/gene_data/GSE125860.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols and extract from step 3 and 6
# Load the gene expression data (already loaded from Step 6)
print(f"Gene data shape before normalization: {gene_data.shape}")

try:
    # Normalize gene symbols using the NCBI Gene database information
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    
    if normalized_gene_data.empty:
        print("Normalization resulted in empty dataframe. Using original gene data instead.")
        normalized_gene_data = gene_data
    
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data to the output file
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene data: {e}. Using original gene data instead.")
    normalized_gene_data = gene_data
    # Save the original gene data if normalization fails
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)

# 2. Link clinical and genetic data
# Use the trait_row identified in Step 2 (trait_row = 1) to extract trait data
is_trait_available = trait_row is not None

if is_trait_available:
    # Extract clinical features using the function and conversion methods from Step 2
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
else:
    # Create a minimal dataframe with just the trait column
    linked_data = pd.DataFrame({trait: [np.nan]})
    print("No trait data available, creating minimal dataframe for validation.")

# 3. Handle missing values in the linked data
if is_trait_available:
    print("\nHandling missing values...")
    linked_data = handle_missing_values(linked_data, trait)
    print(f"After missing value handling, linked data shape: {linked_data.shape}")

# 4. Determine whether trait and demographic features are biased
if is_trait_available and not linked_data.empty and len(linked_data.columns) > 1:
    print("\nEvaluating feature bias...")
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Trait bias evaluation result: {is_biased}")
else:
    is_biased = False
    print("Skipping bias evaluation due to insufficient data.")

# 5. Final validation and save metadata
note = ""
if not is_trait_available:
    note = f"Dataset contains gene expression data but no {trait} measurements."
elif is_biased:
    note = f"Dataset contains {trait} data but its distribution is severely biased."

# Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Save the linked data if usable
print(f"\nDataset usability: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset is not usable for {trait} association studies. Data not saved.")

Gene data shape before normalization: (20033, 173)
Gene data shape after normalization: (20033, 173)


Normalized gene data saved to ../../output/preprocess/Hepatitis/gene_data/GSE125860.csv
Clinical data saved to ../../output/preprocess/Hepatitis/clinical_data/GSE125860.csv
Linked data shape: (176, 20036)

Handling missing values...
After missing value handling, linked data shape: (0, 2)
Skipping bias evaluation due to insufficient data.
Abnormality detected in the cohort: GSE125860. Preprocessing failed.

Dataset usability: False
Dataset is not usable for Hepatitis association studies. Data not saved.
